### Make a surface and domain file given a lat  lon 

In [1]:
from scipy.stats import qmc
import numpy as np
import xarray as xr
import csv
import pandas as pd
import os
import netCDF4 as nc4
import sys
from tempfile import TemporaryFile                                                                                                                                 
import argparse                                                                                                                                                     
import shutil                                                                                                                                                       
import tempfile 
import random


### Domain file

In [2]:
domain_full = '/global/cfs/cdirs/e3sm/inputdata/share/domains/domain.lnd.fv1.9x2.5_gx1v6.090206.nc'
domain_full = xr.open_dataset(domain_full,  decode_times=False)

lon_mids = domain_full.xc[0,0:144]
lat_mids = domain_full.yc[0:96,0]

In [3]:
lat = 9.154300000000
lon = -79.846100000000

if lon < 0 : 
    lon = 360 + lon
    print(lon)

abslat = np.abs(lat_mids - lat)
abslon = np.abs(lon_mids - lon)
c = np.maximum(abslon, abslat)


([xloc],[yloc]) = np.where(c == np.min(c))

lat_new = lat_mids[yloc]
print(lat_new.values)
lon_new = lon_mids[xloc]
print(lon_new.values)

domain = domain_full.sel(ni=xloc,nj=yloc)

280.1539
8.5263157894736
279.99999999999994


#### Set frac to 1

In [4]:
domain['frac'] = 1.0

#### Save domain file

In [5]:
domain.to_netcdf('/pscratch/sd/j/jneedham/fates-tutorial/met_data/domain_bci_fates_tutorial.nc')

### Surface data file

In [6]:
surfdata_full = '/global/cfs/cdirs/e3sm/inputdata/lnd/clm2/surfdata_map/surfdata_1.9x2.5_simyr2000_c180306.nc'
surfdata_full = xr.open_dataset(surfdata_full,  decode_times=False)
surfdata_full.dims

Frozen({'lsmlat': 96, 'lsmlon': 144, 'nlevsoi': 10, 'natpft': 17, 'time': 12, 'lsmpft': 17, 'numurbl': 3, 'numrad': 2, 'nlevurb': 5})

##### Subset by lat and lon

In [7]:
surfdata = surfdata_full.sel(lsmlon=xloc,lsmlat=yloc)

#### Set percent vegetation to 100%

In [8]:
surfdata['PCT_NATVEG'] = 100
new_urban = [0,0,0]
new_urban_array = xr.DataArray(new_urban, dims=['numurbl'], coords = {'numurbl':surfdata['numurbl']})

surfdata['PCT_URBAN'] = new_urban_array
surfdata['PCT_LAKE'] = 0

#### Save surface file

In [9]:
surfdata.to_netcdf('/pscratch/sd/j/jneedham/fates-tutorial/met_data/surfdata_bci_fates_tutorial.nc')